In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import utils
# 載入 CIFAR-10 資料集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

/home/e806/anaconda3/envs/naturalspeech/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [2]:
# 生成器網路
class Generator(nn.Module):
    def __init__(self, latent_dim=100, num_classes=10):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.label_emb = nn.Embedding(num_classes, latent_dim)

        self.model = nn.Sequential(
            nn.Linear(latent_dim * 2, 512 * 2 * 2),
            nn.BatchNorm1d(512 * 2 * 2),
            nn.ReLU(inplace=True),
            nn.Unflatten(1, (512, 2, 2)),

            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, z, labels):
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out

In [3]:
# 判別器網路    
class Discriminator(nn.Module):
    def __init__(self, num_classes=10):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes
        self.label_emb = nn.Embedding(num_classes, 512)

        self.conv1 = utils.spectral_norm(nn.Conv2d(3, 64, 4, stride=2, padding=1))
        self.conv2 = utils.spectral_norm(nn.Conv2d(64, 128, 4, stride=2, padding=1))
        self.conv3 = utils.spectral_norm(nn.Conv2d(128, 256, 4, stride=2, padding=1))

        self.model = nn.Sequential(
            self.conv1,
            nn.LeakyReLU(0.2, inplace=True),

            self.conv2,
            nn.LeakyReLU(0.2, inplace=True),

            self.conv3,
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
        )

        self.out = nn.Sequential(
            nn.Linear(256 * 4 * 4 + 512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        x = self.model(img)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        validity = self.out(x)
        return validity

In [4]:
from tqdm import tqdm
import os
# 訓練開始前建立目錄
os.makedirs("generated_images", exist_ok=True)
# 設定超參數
latent_dim = 100
img_shape = (32, 32, 3)
lr = 0.0002
b1 = 0.5
b2 = 0.999
num_epochs = 200
batch_size = 64

# 初始化生成器和判別器
generator = Generator(latent_dim)
discriminator = Discriminator()

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

# 設定優化器
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

# 定義損失函數
criterion = nn.BCELoss()

# 生成固定的雜訊和標籤,用於生成圖像
fixed_noise = torch.randn(100, latent_dim, device=device)
fixed_labels = torch.randint(0, 10, (100,), device=device)

# 訓練迴圈
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for i, (imgs, labels) in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1}", leave=False)):
        real_imgs = imgs.to(device)
        labels = labels.to(device)
        
        # 訓練判別器
        optimizer_D.zero_grad()
        
        # 真實圖像
        real_validity = discriminator(real_imgs, labels)
        d_real_loss = criterion(real_validity, torch.ones_like(real_validity))
        
        # 生成圖像
        z = torch.randn(batch_size, latent_dim, device=device)
        gen_labels = torch.randint(0, 10, (batch_size,), device=device)
        gen_imgs = generator(z, gen_labels)
        fake_validity = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = criterion(fake_validity, torch.zeros_like(fake_validity))
        
        # 反向傳播與優化
        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
        # 訓練生成器
        optimizer_G.zero_grad()
        
        # 生成圖像
        fake_validity = discriminator(gen_imgs, gen_labels)
        g_loss = criterion(fake_validity, torch.ones_like(fake_validity))
        
        # 反向傳播與優化
        g_loss.backward()
        optimizer_G.step()
        
        # 打印訓練狀態
        if (i+1) % 100 == 0:
            print(f"[Epoch {epoch+1}/{num_epochs}] [Batch {i+1}/{len(trainloader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
    
    # 在每個 epoch 結束時生成並儲存圖像
    with torch.no_grad():
        gen_imgs = generator(fixed_noise, fixed_labels).detach().cpu()
        img_grid = torchvision.utils.make_grid(gen_imgs, nrow=10, normalize=True)
        plt.figure(figsize=(10,10))
        plt.imshow(img_grid.permute(1, 2, 0))
        plt.axis('off')
        plt.title(f"Epoch {epoch+1}")
        plt.savefig(f"generated_images/epoch_{epoch+1}.png", bbox_inches='tight')
        plt.close()

Epochs:   0%|          | 0/200 [00:00<?, ?it/s]

[Epoch 1/200] [Batch 100/1563] [D loss: 0.5180] [G loss: 1.3833]


[Epoch 1/200] [Batch 200/1563] [D loss: 0.8617] [G loss: 1.7376]


[Epoch 1/200] [Batch 300/1563] [D loss: 0.6779] [G loss: 0.8711]


[Epoch 1/200] [Batch 400/1563] [D loss: 0.4955] [G loss: 1.2112]


...

[Epoch 18/200] [Batch 1400/1563] [D loss: 0.4424] [G loss: 1.5333]


[Epoch 18/200] [Batch 1500/1563] [D loss: 0.4021] [G loss: 2.1661]


Epochs:   9%|▉         | 18/200 [03:16<33:30, 11.05s/it]

[Epoch 19/200] [Batch 100/1563] [D loss: 0.4815] [G loss: 1.8780]


[Epoch 19/200] [Batch 200/1563] [D loss: 0.3991] [G loss: 2.1992]
